In [2]:
import numpy as np
import pandas as pd

In [ ]:
dataset_dir = "datasets"
file = 'news_traindata' # 'WELFake_Dataset.csv'
nrows = 10000
vocab_file = f'vocabs/vokab_{file}_{nrows}.pkl'
model_file = f"models/simple_model_{file}_{nrows}.pth"

df = pd.read_csv(f'{dataset_dir}/{file}.csv', encoding='utf-8', nrows=nrows)
df = df.dropna()

df.head(20)

,type,title,content,label
0,junksci,First Certified Organic Fast Food Restaurant t...,by ARIANA MARISOL\n\nFast food restaurants are...,1
1,reliable,UP polls: Parties say BJP setting agenda; vote...,India News UP polls: Parties say BJP setting a...,0
2,hate,UCLA Student Is Charged With Attempted Murder ...,A UCLA student allegedly stabbed a classmate f...,1
4,hate,National Vanguard,Transcript by Katana IT HAS NOW become absolut...,1
5,conspiracy,Truth Broadcast Network,448 Views0 Likes\n\nEurope has again been forc...,1
6,fake,Meet China’s Morlocks: 1 Million Beijing Resid...,Meet China’s Morlocks: 1 Million Beijing Resid...,1
7,conspiracy,The Global Elite’s Digital Agenda Played Out a...,"Susanne Posel, Contributor\n\nActivist Post\n\...",1
8,political,Meet the 99.999999 Percent,Forget about the 1 percent versus the 99 perce...,1
9,satire,Isaiah Thomas praises LeBron James,Advertisements\n\nAdvertisements\n\nEven thoug...,1
10,bias,Poetry’s Plum Gone to Hell,What is the use of writing about books?” asked...,1


## Preprocessing

In [ ]:
import os
import pickle
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from collections import Counter

MAX_VOCAB = 25000
special_tokens = ['<unk>', '<pad>']
tokenizer = get_tokenizer('basic_english')

def generate_vocabulary(df):
    counter = Counter()
    for text in df['content']:
        if pd.notna(text):
            counter.update(tokenizer(str(text)))
    
    most_common = [token for token, _ in counter.most_common(MAX_VOCAB - len(special_tokens))]
    
    vocab = build_vocab_from_iterator([most_common], specials=special_tokens)
    vocab.set_default_index(vocab['<unk>'])

    with open(vocab_file, 'wb') as f:
        pickle.dump(vocab, f)
    print(f"Vocabulary saved to '{vocab_file}'.")

    return vocab


if os.path.exists(vocab_file):
    with open(vocab_file, 'rb') as f:
        vocab = pickle.load(f)
    print("Vocabulary loaded from 'vocab.pkl'.")

else:
    vocab = generate_vocabulary(df)


    
    


Vocabulary loaded from 'vocab.pkl'.


In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F

MAX_LENGTH = 2048

# Label encode the 'type' column
le = LabelEncoder()
df['type_encoded'] = le.fit_transform(df['type'])

NUM_CLASSES = len(le.classes_)  # Number of unique classes

encoded_texts_and_labels = []
for text, label in zip(df['content'], df['type_encoded']):
    if pd.notna(text):
        encoded = [vocab[token] for token in tokenizer(text)]
        
        if len(encoded) <= MAX_LENGTH:
            encoded_texts_and_labels.append((torch.tensor(encoded, dtype=torch.long), label))

# Separate encoded texts and labels
encoded_texts = [item[0] for item in encoded_texts_and_labels]
label_indices = torch.tensor([item[1] for item in encoded_texts_and_labels], dtype=torch.long)

# Create one-hot encoded labels
labels = F.one_hot(label_indices, num_classes=NUM_CLASSES).float()

# Pad sequences
padded_texts = pad_sequence(encoded_texts, batch_first=True, padding_value=vocab['<pad>'])

print(f"Filtered texts: {len(padded_texts)}, Labels: {labels.shape}")


Filtered texts: 9668, Labels: torch.Size([9668, 11])


In [7]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


class NewsDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

X_train, X_val, y_train, y_val = train_test_split(padded_texts, labels, test_size=0.1)

train_ds = NewsDataset(X_train, y_train)
val_ds = NewsDataset(X_val, y_val)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, pin_memory=True, num_workers=0)
val_dl = DataLoader(val_ds, batch_size=32, num_workers=0)

In [16]:
from mulstage_model import CNN_BiLSTM
from torch import nn


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")
model = CNN_BiLSTM(vocab=vocab, vocab_size=len(vocab), embed_dim=100, hidden_dim=128, output_dim=NUM_CLASSES, pad_idx=vocab['<pad>'])
model.to(device)

print("done constructing model")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)   

print("done constructing optimizer")

from tqdm import tqdm

def train(model, loader):
    model.train()
    total_loss = 0
    for xb, yb in tqdm(loader, desc="Training", leave=False):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    return total_loss / len(loader)

def evaluate(model, loader):
    model.eval()
    total_acc = 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            # print(preds.shape, yb.shape)
            preds_class = preds.argmax(dim=1)
            #v print(preds_class)
            labels_class = yb.argmax(dim=1)
            # print(labels_class)
            total_acc += (preds_class == labels_class).float().mean().item()
            # print(preds_class, labels_class, total_acc)
    return total_acc / len(loader)

print("training")

for epoch in range(5):
    loss = train(model, train_dl)
    acc = evaluate(model, val_dl)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Val Acc: {acc:.4f}")


Using device: mps
done constructing model
done constructing optimizer
training


Epoch 1, Loss: 2.1447, Val Acc: 0.5811


Epoch 2, Loss: 1.9312, Val Acc: 0.6285


Epoch 3, Loss: 1.8641, Val Acc: 0.6617


Epoch 4, Loss: 1.8280, Val Acc: 0.6668


Epoch 5, Loss: 1.7975, Val Acc: 0.7087


In [17]:
# torch.save(model.state_dict(), f"model_{file}_{nrows}.pth")
for epoch in range(5):
    loss = train(model, train_dl)
    acc = evaluate(model, val_dl)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Val Acc: {acc:.4f}")

Epoch 1, Loss: 1.7678, Val Acc: 0.6990


Epoch 2, Loss: 1.7472, Val Acc: 0.6869


Epoch 3, Loss: 1.7309, Val Acc: 0.7117


Epoch 4, Loss: 1.7138, Val Acc: 0.7117


Epoch 5, Loss: 1.7025, Val Acc: 0.7147


In [21]:

test_acc = evaluate(model, val_dl)
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.9755
